# Clean Raw Data 

- This script cleans the raw data output from psychopy
- raw data is available via google drive per request

In [1]:
import json
#Pandas for saving datasets
import pandas as pd
#matplotlib for rendering
import matplotlib.pyplot as plt
#numpy for handeling matrix operations
import numpy as np
#time, to, well... keep track of time
import time
#iPython display for making sure we can render the frames
from IPython import display
#seaborn for rendering
import seaborn
import os

In [2]:
import ast

# Prepare Data #

In [3]:
"""
all_col_names = ['set1','set2',"run_tr_btn.started", "run_tr_btn.rt", "ex_video_Onset", "external_bpress.rt", "external_bpress.started", 
                 "fixation_cross.started", 'tr_external.rt', "tr_external.started", "in_video_Onset",
                 "internal_bpress.rt","internal_bpress.started","internal_TR.rt"]
to_clean_col = ['run_tr_btn.rt', 'external_bpress.rt','tr_external.rt', "internal_bpress.rt", "internal_TR.rt"]

other_col = ['set1','set2',"run_tr_btn.started","fixation_cross.started","ex_video_Onset", "external_bpress.started", "tr_external.started",
             "in_video_Onset","internal_bpress.started"]
"""

'\nall_col_names = [\'set1\',\'set2\',"run_tr_btn.started", "run_tr_btn.rt", "ex_video_Onset", "external_bpress.rt", "external_bpress.started", \n                 "fixation_cross.started", \'tr_external.rt\', "tr_external.started", "in_video_Onset",\n                 "internal_bpress.rt","internal_bpress.started","internal_TR.rt"]\nto_clean_col = [\'run_tr_btn.rt\', \'external_bpress.rt\',\'tr_external.rt\', "internal_bpress.rt", "internal_TR.rt"]\n\nother_col = [\'set1\',\'set2\',"run_tr_btn.started","fixation_cross.started","ex_video_Onset", "external_bpress.started", "tr_external.started",\n             "in_video_Onset","internal_bpress.started"]\n'

In [4]:
"""
"internal_TR.rt" is deleted for this set -- do not need unless needed to check other TR ish 
"""
all_col_names = ['set1','set2',"run_tr_btn.started", "run_tr_btn.rt", "ex_video_Onset", "external_bpress.rt", "external_bpress.started", 
                 "fixation_cross.started", 'tr_external.rt', "tr_external.started", "in_video_Onset",
                 "internal_bpress.rt","internal_bpress.started"]
to_clean_col = ['run_tr_btn.rt', 'external_bpress.rt','tr_external.rt', "internal_bpress.rt"]

other_col = ['set1','set2',"run_tr_btn.started","fixation_cross.started","ex_video_Onset", "external_bpress.started", "tr_external.started",
             "in_video_Onset","internal_bpress.started"]


In [5]:
## subject notes ###
"""
~~~~ GENERAL ~~~~
sub-015 and sub-001 are the only subjects that cannot run with code below
fMRI: sub-029 initially excluded cuz scanner malfunction



sub-001: skipped cuz split into two files ***To Do ***
sub-002: NO internal bpress RT, but there are still counts for button presses. External video is fine
    - Extra row observed for external, which was manually deleted. Should be fine to rerun -- i removed from bad_subs
    - i added a column in the excel doc so that it would run. but the timepoints for bpress onset are sham
    - additionally i deleted the TR reaction time from the 'to_clean_col' variable, which did not exist in the doc either.
    - *should be fine for rerunning ** as long as the 'Internal_TR.rt' is deleted
sub-003: Extra row observed for external, which was manually deleted. Should be fine to rerun -- i removed from bad_subs
    - current brushing: 87, TRUE: 88 for THREE runs. last run when there was an issue and scan had to be restarted
    - Scan 6 == 1 run of brushing
    - Scan 7 = 3 runs of brushing 
    - ** no idea when run 7 was started tho! will prolly need to throw out! For now we can process all
sub-012: issue with external shrek.. short one TR... --should be fixed now with new check_d code
    - 7 scans -- discard 6, use 7
sub-014: NO internal OR external bpress, curious! ---should be fixed now
    - changed excel data to include two null columns
    - ALSO only 30 TRs for the second run epi data. i went into the room to adjust runny eyes. Data for this run is not usable. but after should
      be okay if u want to put in the effort 
sub-015: less than 24 runs ---should be fixed now w custom code (but keep out of main run cuz will throw an error otherwise )
    - subject is fuqd -- messup during the office with scanning
        - DO NOT use behavioral data for third run of the office - this is where it died. 
        - Fourth run okay...? yea looks like there is data
    - also did not complete internal CAKE, so only five runs. Data file is cleaned though
    - SUMMARY: need to exclude the office run 3 for fMRI but can include run 3 in behavioral, no data at all for CAKE
    - 7 runs:
    - **** Use external, ignore internal for now... cuz internal is fuqd. Can preprocess all 7 tho
    
sub-020: NO internal OR external bpress, curious! ---should be fixed now
    - changed excel data to include two null columns
    - this sub and sub-014 are craaaazzzy. sub-014 had runny eyes and told me after that they never mind wandered. idk about 20!
sub-029 length issue: current cake: 98, TRUE: 99 --should be fixed now with new check_d code
    - scanner malfunction! Exclude initially! 
sub-032: fixed!
    - copied data from first run into the second session and saved as _MERGED.csv. ran with no problems -- although there is potentially
      some weird timing stuff going on with the first run...
sub-037: NO internal OR external bpress, curious! ---should be fixed now
    - changed excel data to include two null columns
    - according to excel was a solid sub...?

~~ Notes ~~
one-off errors are due to a rounding issue. Sometimes we need to round up 1.5 instead of to the nearest whole number. 
- 


"""

"\n~~~~ GENERAL ~~~~\nsub-015 and sub-001 are the only subjects that cannot run with code below\nfMRI: sub-029 initially excluded cuz scanner malfunction\n\n\n\nsub-001: skipped cuz split into two files ***To Do ***\nsub-002: NO internal bpress RT, but there are still counts for button presses. External video is fine\n    - Extra row observed for external, which was manually deleted. Should be fine to rerun -- i removed from bad_subs\n    - i added a column in the excel doc so that it would run. but the timepoints for bpress onset are sham\n    - additionally i deleted the TR reaction time from the 'to_clean_col' variable, which did not exist in the doc either.\n    - *should be fine for rerunning ** as long as the 'Internal_TR.rt' is deleted\nsub-003: Extra row observed for external, which was manually deleted. Should be fine to rerun -- i removed from bad_subs\n    - current brushing: 87, TRUE: 88 for THREE runs. last run when there was an issue and scan had to be restarted\n    - Sc

In [6]:
## TR length of each movie ## 
mov_len_dic = {
'oragami' :  82,
'shrek' : 90,
'sherlock' : 98,
'brushing' : 88,
'cake' : 99,
'office' : 102    
}

In [7]:
behav_dir = '/Users/isaacchristian/Desktop/Princeton/RESEARCH/MEI/DATA/behav/raw/'
out_dir = '/Users/isaacchristian/Desktop/Princeton/RESEARCH/MEI/DATA/behav/clean/'
cod_dir = '/Users/isaacchristian/Desktop/Princeton/RESEARCH/MEI/code/'
sub = 'sub-007_MEI-3-15-23_lag_onevid_eye_2023_Mar_28_1648_1.csv'

In [9]:
file_list = sorted(os.listdir(behav_dir))[1:]
#np.save(cod_dir+'/file_list.npy', file_list)

In [10]:
subset_files = file_list[2:15] + file_list[16:]

In [11]:
subset_files

['sub-002_MEI-3-15-23_lag_onevid_eye_2023_Mar_21_1752_1.csv',
 'sub-003_MEI-3-15-23_lag_onevid_eye_2023_Mar_23_1345.csv',
 'sub-004_MEI-3-15-23_lag_onevid_eye_2023_Mar_24_0947_1.csv',
 'sub-005_MEI-3-15-23_lag_onevid_eye_2023_Mar_24_1306.csv',
 'sub-006_MEI-3-15-23_lag_onevid_eye_2023_Mar_28_1241_1.csv',
 'sub-007_MEI-3-15-23_lag_onevid_eye_2023_Mar_28_1648_1.csv',
 'sub-008_MEI-3-15-23_lag_onevid_eye_2023_Mar_29_1642_1.csv',
 'sub-009_MEI-3-15-23_lag_onevid_eye_2023_Mar_30_1336_1.csv',
 'sub-010_MEI-3-15-23_lag_onevid_eye_2023_Mar_30_1611_1.csv',
 'sub-011_MEI-3-15-23_lag_onevid_eye_2023_Mar_31_1025_1.csv',
 'sub-012_MEI-3-15-23_lag_onevid_eye_2023_Mar_31_1234_1.csv',
 'sub-013_MEI-3-15-23_lag_onevid_eye_2023_Mar_31_1532_1.csv',
 'sub-014_MEI-3-15-23_lag_onevid_eye_2023_Apr_03_1148_1.csv',
 'sub-016_MEI-3-15-23_lag_onevid_eye_2023_Apr_05_1634_1.csv',
 'sub-017_MEI-3-15-23_lag_onevid_eye_2023_Apr_06_1611_1.csv',
 'sub-018_MEI-3-15-23_lag_onevid_eye_2023_Apr_07_1255_1.csv',
 'sub-019_ME

In [12]:
bad_subs = file_list[:4] +  [file_list[15]] +   file_list[32:34] + [file_list[40]]

In [13]:
bad_subs

['sub-001_MEI-3-15-23_lag_onevid_eye_2023_Mar_21_1531_run1.csv',
 'sub-001_MEI-3-15-23_lag_onevid_eye_2023_Mar_21_1553_runs2+3.csv',
 'sub-002_MEI-3-15-23_lag_onevid_eye_2023_Mar_21_1752_1.csv',
 'sub-003_MEI-3-15-23_lag_onevid_eye_2023_Mar_23_1345.csv',
 'sub-015_MEI-3-15-23_lag_onevid_eye_2023_Apr_05_1050.csv',
 'sub-032_MEI-3-15-23_lag_onevid_eye_2023_MERGED.csv',
 'sub-033_MEI-3-15-23_lag_onevid_eye_2023_May_01_1027_1.csv',
 'sub-040_MEI-3-15-23_lag_onevid_eye_2023_May_11_1326_1.csv']

In [14]:
file_list[15]

'sub-015_MEI-3-15-23_lag_onevid_eye_2023_Apr_05_1050.csv'

# Filter the dataframe 

In [8]:
def clean_file(data):
    # Fill all NA values with
    data1 = data.fillna("-1")

    # Replace runs with no button presses with a "[]"
    data1 = data1.replace('None', "[]")

    ## only keep data where movie was played
    data1 = data1[data1.set1 != "-1"]
    
    # extract columns that need integer conversion # 
    df_to_clean = data1[to_clean_col]
    # extract other columns to append
    df_good = data1[other_col]
    ## convert strings to arrays for bad columns ## 
    ## make it so lists read out from file are accessible ##
    for col in to_clean_col:
        df_to_clean[col] = df_to_clean[col].apply(ast.literal_eval)
    ## combine all columns and convert to list ## 
    df_filt = pd.concat([df_good, df_to_clean],axis=1)
    df_filt_list = df_filt.to_dict('list')
    ## check that this file is good ##
    assert len(df_filt) == 24, "were there less than 24 runs, if manully edit haha"
    ## return cleaned dframe ## 
    return df_filt_list, df_filt


# check dimensions of each run 

In [9]:
def check_d(mov_name, info_dic, movie_dic):
    """
    This function ensures that the current calculated TR length from video onset to start fixation is equal
    to the known video length
    If there is a one off error due to rounding, fix, otherwise, report the error is not fixed
    Input:
     - mov_name: the current video (ie sherlock)
     - info_dic: the current start/end tr info: (ie {'start_tr': 319, 'end_tr': 417, 'bpress': [143.25608371198177]})
     - movie_dic: the known length of each movie
    Returns:
     - the checked dictionary or the fixed dictionary if length is wrong
    """
    
    # Get the current movie length # 
    cur_mov_len = int(np.ceil(info['end_tr'] - info['start_tr']))
    # Return the dictionary if lenght matches known movie length
    if cur_mov_len == int(movie_dic[mov_name]): 
        return info_dic
    else: 
        # Fix movie length if necessary # 
        print('WARNING, NOT THE SAME MOVIE LENGTH')
        print(f'current {mov_name}: {cur_mov_len}, TRUE: {int(movie_dic[mov_name])}')
        #print(cur_mov_len - int(movie_dic[mov_name]))
        if (cur_mov_len - int(movie_dic[mov_name]) == -1):
            print('end TR begin', info_dic['end_tr'])
            ## add one TR ## 
            info_dic['end_tr'] = info['end_tr'] +1
            print('end TR END', info_dic['end_tr'])
            cur_mov_len = int(np.ceil(info['end_tr'] - info['start_tr']))
        ## Check to see if probleme was fixed ## 
        if cur_mov_len == int(movie_dic[mov_name]): 
            print('FIXED')
        else:
            print('NOT FIXED SHEEESH')
            
        return info_dic

# Label Dictionary # 

In [17]:
# Find meta-data about the order of movie presentation # 

# How many vids for each condition? # 
num_vids = 12

def met_dat(df_filt, num_vids, sub_num):
    """
    Find meta-data about the order of movie presentation # 
    input:
    - df_filt: filtered pandas dataframe
    - num_vids: number of videos for each condition
    returns:
    - Dictionary consisting of information about the order of movie presentation
    """
    ### ODD ## 
    print(f'subnum is {sub_num}')
    if sub_num % 2 == 1:
        cond = np.concatenate((np.repeat('External',num_vids),np.repeat('Internal',num_vids)))
        # switch set 1 and set two ordering depending on internal or external conditions ie :12
        # set 2 is external, set 1 is set to be internal
        mov_name_ex = df_filt['set2'][:12].str[11:]
        mov_name_in = df_filt['set1'][12:].str[11:]
        movie_present = np.hstack((mov_name_ex,mov_name_in))
        assert len(movie_present) == 24, "check movie naming"
    else:
        cond = np.concatenate((np.repeat('Internal',num_vids),np.repeat('External',num_vids)))
        # switch set 1 and set two ordering depending on internal or external conditions ie :12
        mov_name_ex = df_filt['set2'][12:].str[11:]
        mov_name_in = df_filt['set1'][:12].str[11:]
        movie_present = np.hstack((mov_name_in, mov_name_ex))
        assert len(movie_present) == 24, "check movie naming"

    vid_num = np.arange(1,num_vids*2+1)
    
    ## create label dic ## 
    label_dic = {'vid_num': vid_num, 'condition': cond, 'movies': movie_present}
    ## create dictionary describing movie presentation order ## 
    
    temp_df = pd.DataFrame(movie_present, columns = ['prez']).prez.unique()
    
    #print(temp_df.prez.unique())
    mov_prez = [mov[:-9] for mov in temp_df]
    #mov_prez = {}
    #for ind, mov in enumerate(temp_df2):
     #   mov_prez[ind+1] = mov[:-9] 
        
    return label_dic, mov_prez
    

# Run 

In [18]:
#x = np.unique(label_dic['movies'], return_index=True)

In [10]:
# How many TRs should we add to the beginning of each fMRI run? -- its three, see below comments # 
add_tr = 3

# # How many TRs should we append to the end of the video to capture hemo lag? # 
end_add_tr = 4

# TR
tr = 1.5

In [24]:
subset_files[29:30]

['sub-032_MEI-3-15-23_lag_onevid_eye_2023_MERGED.csv']

In [35]:
#for file in subset_files[1:2]:
for file in subset_files[10:11]:
    data = pd.read_csv(behav_dir+file)
    sub, sub_num = file[:7], int(file[4:7])
    print(sub)
    df_filt_dic, df_filt_pd = clean_file(data)
    print(f'{sub} is clean!')
    ## Create dictionary denoting order of movie appearance # 
    label_dic, mov_prez = met_dat(df_filt_pd, num_vids, sub_num)
    print(f'{sub} label dicc created')
    ## check that we have 6 runs ## 
    assert len(np.where(np.asarray(df_filt_dic['run_tr_btn.started'])!="-1")[0]) == 6, "more than 6 runs!"


    ### dix initialization ## 
    vid_dic = {}
    run_dic = {}
    cond_dic = {}
    sub_dic = {}
    
    ## Run 
    """ 
    df_filt_dic['run_tr_btn.started'] is the time that the fourth TR **started** for each of the 6 runs.
    So, I need to add 3 TRs to the start and end tr so that the index aligns with the epi data
    """
    for ind, vid in enumerate(df_filt_dic['run_tr_btn.started']):
        # this is a new video # 
        # set start time for the current run ## 
        ## If new video set the 
        if vid != "-1": 
            start_time = vid
            run_dic = {}
            run = 1
        # Set dictionary key to the current name of the movie after four iterations
        if ((ind + 1) % 4 == 0): 
            cond_dic[label_dic['movies'][ind][:-9]] = run_dic

        # set movie name for each iteration
        mov_name = label_dic['movies'][ind][:-9]

        # Set dictionary key to internal or external
        if ((ind + 1) % 12 == 0): 
            sub_dic[label_dic['condition'][ind]] = cond_dic
            cond_dic = {}

        # Check external to see if the current index is null # 
        if df_filt_dic['ex_video_Onset'][ind] != '-1':
            # Start 
            vid_start = df_filt_dic['ex_video_Onset'][ind] - start_time

            # End # 
            vid_end = df_filt_dic['fixation_cross.started'][ind] - df_filt_dic['ex_video_Onset'][ind]

            # Dictionary # 
            info = { 
                "start_tr" : int(np.ceil(vid_start/tr) + add_tr),
                "end_tr" : int(np.ceil(np.asarray(vid_start/tr) + np.asarray(vid_end/tr)) + add_tr),
                "bpress" : df_filt_dic['external_bpress.rt'][ind]
            }
            info_checked = check_d(mov_name, info, mov_len_dic)
            ## append TRs to the end of the run
            info_checked["end_tr"] = info_checked["end_tr"] + end_add_tr
            ## save information for the run 
            run_dic['run-'+str(run)] = info_checked
            run+=1

        else:
            # round, add 4 TRs to account for magnet stablization
            vid_start = df_filt_dic['in_video_Onset'][ind] - start_time

            # End # 
            vid_end = df_filt_dic['fixation_cross.started'][ind] - df_filt_dic['in_video_Onset'][ind]

            # Dictionary # 
            info = { 
                "start_tr" : int(np.ceil(vid_start/tr) + add_tr),
                "end_tr" : int(np.ceil(np.asarray(vid_start/tr) + np.asarray(vid_end/tr)) + add_tr),
                "bpress" : df_filt_dic['internal_bpress.rt'][ind]
            }
            # check that movie dimensions are correct before continuing 
            info_checked = check_d(mov_name, info, mov_len_dic)
            ## append TRs to the end of the run
            info_checked["end_tr"] = info_checked["end_tr"] + end_add_tr
            ## save information for the run 
            run_dic['run-'+str(run)] = info_checked
            run+=1

    sub_dic['mov_order'] = mov_prez
    ##### Save ##
    #np.save(f'{out_dir}{sub}_behav.npy', sub_dic)
    
    
    print()
   
    
    

sub-012
sub-012 is clean!
subnum is 12
sub-012 label dicc created
WARNING, NOT THE SAME MOVIE LENGTH
current shrek: 89, TRUE: 90
end TR begin 183
end TR END 184
FIXED
WARNING, NOT THE SAME MOVIE LENGTH
current shrek: 89, TRUE: 90
end TR begin 277
end TR END 278
FIXED
WARNING, NOT THE SAME MOVIE LENGTH
current shrek: 89, TRUE: 90
end TR begin 371
end TR END 372
FIXED
WARNING, NOT THE SAME MOVIE LENGTH
current shrek: 89, TRUE: 90
end TR begin 467
end TR END 468
FIXED



<ipython-input-15-75f5a3f335e6>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_clean[col] = df_to_clean[col].apply(ast.literal_eval)


# Single Sub sorting into conditions

In [84]:
def clean_file(data):
    # Fill all NA values with
    data1 = data.fillna("-1")

    # Replace runs with no button presses with a "[]"
    data1 = data1.replace('None', "[]")

    ## only keep data where movie was played
    data1 = data1[data1.set1 != "-1"]
    
    # extract columns that need integer conversion # 
    df_to_clean = data1[to_clean_col]
    # extract other columns to append
    df_good = data1[other_col]
    ## convert strings to arrays for bad columns ## 
    ## make it so lists read out from file are accessible ##
    for col in to_clean_col:
        df_to_clean[col] = df_to_clean[col].apply(ast.literal_eval)
    ## combine all columns and convert to list ## 
    df_filt = pd.concat([df_good, df_to_clean],axis=1)
    df_filt_list = df_filt.to_dict('list')
    ## check that this file is good ##
    print(len(df_filt))
    assert len(df_filt) == 24, "were there less than 24 runs, if manully edit haha"
    ## return cleaned dframe ## 
    return df_filt_list, df_filt


In [182]:
info_checked

{'start_tr': 319, 'end_tr': 417, 'bpress': [143.25608371198177]}

# check dimensions of each run 

In [196]:
def check_d(mov_name, info_dic, movie_dic):
    """
    This function ensures that the current calculated TR length from video onset to start fixation is equal
    to the known video length
    If there is a one off error due to rounding, fix, otherwise, report the error is not fixed
    Input:
     - mov_name: the current video (ie sherlock)
     - info_dic: the current start/end tr info: (ie {'start_tr': 319, 'end_tr': 417, 'bpress': [143.25608371198177]})
     - movie_dic: the known length of each movie
    Returns:
     - the checked dictionary or the fixed dictionary if length is wrong
    """
    
    # Get the current movie length # 
    cur_mov_len = int(np.ceil(info['end_tr'] - info['start_tr']))
    # Return the dictionary if lenght matches known movie length
    if cur_mov_len == int(movie_dic[mov_name]): 
        return info_dic
    else: 
        # Fix movie length if necessary # 
        print('WARNING, NOT THE SAME MOVIE LENGTH')
        print(f'current {mov_name}: {cur_mov_len}, TRUE: {int(movie_dic[mov_name])}')
        #print(cur_mov_len - int(movie_dic[mov_name]))
        if (cur_mov_len - int(movie_dic[mov_name]) == -1):
            print('end TR begin', info_dic['end_tr'])
            ## add one TR ## 
            info_dic['end_tr'] = info['end_tr'] +1
            print('end TR END', info_dic['end_tr'])
            cur_mov_len = int(np.ceil(info['end_tr'] - info['start_tr']))
        ## Check to see if probleme was fixed ## 
        if cur_mov_len == int(movie_dic[mov_name]): 
            print('FIXED')
        else:
            print('NOT FIXED SHEEESH')
            
        return info_dic
        
    
    

# Label Dictionary # 

In [99]:
# Find meta-data about the order of movie presentation # 

# How many vids for each condition? # 
num_vids = 12

def met_dat(df_filt, num_vids, sub_num):
    """
    Find meta-data about the order of movie presentation # 
    input:
    - df_filt: filtered pandas dataframe
    - num_vids: number of videos for each condition
    returns:
    - Dictionary consisting of information about the order of movie presentation
    """
    ### ODD ## 
    print(f'subnum is {sub_num}')
    if sub_num % 2 == 1:
        cond = np.concatenate((np.repeat('External',num_vids),np.repeat('Internal',num_vids)))
        # switch set 1 and set two ordering depending on internal or external conditions ie :12
        # set 2 is external, set 1 is set to be internal
        mov_name_ex = df_filt['set2'][:12].str[11:]
        mov_name_in = df_filt['set1'][12:].str[11:]
        movie_present = np.hstack((mov_name_ex,mov_name_in))
        assert len(movie_present) == 24, "check movie naming"
    else:
        cond = np.concatenate((np.repeat('Internal',num_vids),np.repeat('External',num_vids)))
        # switch set 1 and set two ordering depending on internal or external conditions ie :12
        mov_name_ex = df_filt['set2'][12:].str[11:]
        mov_name_in = df_filt['set1'][:12].str[11:]
        movie_present = np.hstack((mov_name_in, mov_name_ex))
        assert len(movie_present) == 24, "check movie naming"

    vid_num = np.arange(1,num_vids*2+1)

    label_dic = {'vid_num': vid_num, 'condition': cond, 'movies': movie_present}
    return label_dic
    

In [ ]:
## for subject 15 who has one condition less ## 

In [62]:
# Find meta-data about the order of movie presentation # 

# How many vids for each condition? # 
num_vids = 10

def met_dat_015(df_filt, num_vids, sub_num):
    """
    Find meta-data about the order of movie presentation # 
    input:
    - df_filt: filtered pandas dataframe
    - num_vids: number of videos for each condition
    returns:
    - Dictionary consisting of information about the order of movie presentation
    """
    ### ODD ## 
    print(f'subnum is {sub_num}')
    if sub_num % 2 == 1:
        cond = np.concatenate((np.repeat('External',12),np.repeat('Internal',8)))
        # switch set 1 and set two ordering depending on internal or external conditions ie :12
        # set 2 is external, set 1 is set to be internal
        mov_name_ex = df_filt['set2'][:12].str[11:]
        
        ### changed so only two videos!!! ## 
        mov_name_in = df_filt['set1'][12:20].str[11:]
        movie_present = np.hstack((mov_name_ex,mov_name_in))
        #assert len(movie_present) == 20, "check movie naming"
    else:
        print('wrong sub')

    vid_num = np.arange(1,num_vids*2+1)

    label_dic = {'vid_num': vid_num, 'condition': cond, 'movies': movie_present}
    return label_dic
    

# Start 

In [ ]:
## subject notes ###
"""
sub-001: skipped cuz split into two files 
sub-002: NO internal bpress RT, but there are still counts for button presses. External video is fine
    - Extra row observed for external, which was manually deleted. Should be fine to rerun -- i removed from bad_subs
    - i added a column in the excel doc so that it would run. but the timepoints for bpress onset are sham
    - additionally i deleted the TR reaction time from the 'to_clean_col' variable, which did not exist in the doc either.
    - *should be fine for rerunning ** as long as the 'Internal_TR.rt' is deleted
sub-003: Extra row observed for external, which was manually deleted. Should be fine to rerun -- i removed from bad_subs
    - current brushing: 87, TRUE: 88 for THREE runs. last run when there was an issue and scan had to be restarted
    - see notes 
sub-012: issue with external shrek.. short one TR... --should be fixed now with new check_d code
sub-014: NO internal OR external bpress, curious! ---should be fixed now
    - changed excel data to include two null columns
sub-015: less than 24 runs
sub-020: no internal... / external bpress?
sub-029 length issue: current cake: 98, TRUE: 99 --should be fixed now with new check_d code
sub-032: lots of ish messed up obviously, two files 
sub-037: no internal... / external bpress?

~~ Notes ~~
one-off errors are due to a rounding issue. Sometimes we need to round up 1.5 instead of to the nearest whole number. 
- 


"""

In [37]:
bad_subs

['sub-001_MEI-3-15-23_lag_onevid_eye_2023_Mar_21_1531_run1.csv',
 'sub-001_MEI-3-15-23_lag_onevid_eye_2023_Mar_21_1553_runs2+3.csv',
 'sub-002_MEI-3-15-23_lag_onevid_eye_2023_Mar_21_1752_1.csv',
 'sub-003_MEI-3-15-23_lag_onevid_eye_2023_Mar_23_1345.csv',
 'sub-014_MEI-3-15-23_lag_onevid_eye_2023_Apr_03_1148_1.csv',
 'sub-015_MEI-3-15-23_lag_onevid_eye_2023_Apr_05_1050.csv',
 'sub-020_MEI-3-15-23_lag_onevid_eye_2023_Apr_12_1635_1.csv',
 'sub-032_MEI-3-15-23_lag_onevid_eye_2023_Apr_28_0940.csv',
 'sub-032_MEI-3-15-23_lag_onevid_eye_2023_May_01_1406_1.csv',
 'sub-039_MEI-3-15-23_lag_onevid_eye_2023_May_05_1222_1.csv']

In [167]:
all_col_names = ['set1','set2',"run_tr_btn.started", "run_tr_btn.rt", "ex_video_Onset", "external_bpress.rt", "external_bpress.started", 
                 "fixation_cross.started", 'tr_external.rt', "tr_external.started", "in_video_Onset",
                 "internal_bpress.rt","internal_bpress.started"]
to_clean_col = ['run_tr_btn.rt', 'external_bpress.rt','tr_external.rt', "internal_bpress.rt"]

other_col = ['set1','set2',"run_tr_btn.started","fixation_cross.started","ex_video_Onset", "external_bpress.started", "tr_external.started",
             "in_video_Onset","internal_bpress.started"]


In [137]:
file = file_list[32:33][0]
file

'sub-032_MEI-3-15-23_lag_onevid_eye_2023_MERGED.csv'

In [138]:
data = pd.read_csv(behav_dir+file)
sub, sub_num = file[:7], int(file[4:7])
print(sub)
df_filt_dic, df_filt_pd = clean_file(data)
print(f'{sub} is clean!')
## Create dictionary denoting order of movie appearance # 
label_dic = met_dat(df_filt_pd, num_vids, sub_num)
print(f'{sub} label dicc created')
## check that we have 6 runs ## 
#assert len(np.where(np.asarray(df_filt_dic['run_tr_btn.started'])!="-1")[0]) == 6, "more than 6 runs!"


### dix initialization ## 
vid_dic = {}
run_dic = {}
cond_dic = {}
sub_dic = {}

sub-032
24
sub-032 is clean!
subnum is 32
sub-032 label dicc created


<ipython-input-84-a7dd01ee9639>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_clean[col] = df_to_clean[col].apply(ast.literal_eval)


In [139]:
label_dic

{'vid_num': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24]),
 'condition': array(['Internal', 'Internal', 'Internal', 'Internal', 'Internal',
        'Internal', 'Internal', 'Internal', 'Internal', 'Internal',
        'Internal', 'Internal', 'External', 'External', 'External',
        'External', 'External', 'External', 'External', 'External',
        'External', 'External', 'External', 'External'], dtype='<U8'),
 'movies': array(['cake_edit.mp4', 'cake_edit.mp4', 'cake_edit.mp4', 'cake_edit.mp4',
        'brushing_edit.mp4', 'brushing_edit.mp4', 'brushing_edit.mp4',
        'brushing_edit.mp4', 'office_edit.mp4', 'office_edit.mp4',
        'office_edit.mp4', 'office_edit.mp4', 'shrek_edit.mp4',
        'shrek_edit.mp4', 'shrek_edit.mp4', 'shrek_edit.mp4',
        'sherlock_edit.mp4', 'sherlock_edit.mp4', 'sherlock_edit.mp4',
        'sherlock_edit.mp4', 'oragami_edit.mp4', 'oragami_edit.mp4',
        'oragami_edit.mp4', '

In [141]:
assert len(np.where(np.asarray(df_filt_dic['run_tr_btn.started'])!="-1")[0]) == 6, "more than 6 runs!"

# Num adjustment # 
add_tr = 3

# TR
tr = 1.5

### dframes and dix ## 
vid_dic = {}
run_dic = {}
cond_dic = {}
sub_dic ={}

In [142]:
""" 
df_filt_dic['run_tr_btn.started'] is the time that the fourth TR **started** for each of the 6 runs.
So, I need to add 3 TRs to the initial start time to adjust for each run 
"""
for ind, vid in enumerate(df_filt_dic['run_tr_btn.started']):
    # this is a new video # 
    # set start time for the current run ## 
    ## If new video set the 
    if vid != "-1": 
        start_time = vid
        run_dic = {}
        run = 1
    # Set dictionary key to the current name of the movie after four iterations
    if ((ind + 1) % 4 == 0): 
        cond_dic[label_dic['movies'][ind][:-9]] = run_dic
    
    # set movie name for each iteration
    mov_name = label_dic['movies'][ind][:-9]
    
    # Set dictionary key to internal or external
    if ((ind + 1) % 12 == 0): 
        sub_dic[label_dic['condition'][ind]] = cond_dic
        cond_dic = {}
                
    # Check external to see if the current index is null # 
    if df_filt_dic['ex_video_Onset'][ind] != '-1':
        # Start 
        vid_start = df_filt_dic['ex_video_Onset'][ind] - start_time
    
        # End # 
        vid_end = df_filt_dic['fixation_cross.started'][ind] - df_filt_dic['ex_video_Onset'][ind]
        #print(f'{mov_name} starts {vid_start} and \n ends at {vid_end}')
        
        # Dictionary # 
        info = { 
            "start_tr" : int(np.ceil(vid_start/tr) + add_tr),
            "end_tr" : int(np.ceil(np.asarray(vid_start/tr) + np.asarray(vid_end/tr)) + add_tr),
            "bpress" : df_filt_dic['external_bpress.rt'][ind]
        }
        info_checked = check_d(mov_name, info, mov_len_dic)
        run_dic['run-'+str(run)] = info_checked
        run+=1
        
    else:
        # round, add 4 TRs to account for magnet stablization
        vid_start = df_filt_dic['in_video_Onset'][ind] - start_time
        
        # End # 
        vid_end = df_filt_dic['fixation_cross.started'][ind] - df_filt_dic['in_video_Onset'][ind]
        
        # Dictionary # 
        info = { 
            "start_tr" : int(np.ceil(vid_start/tr) + add_tr),
            "end_tr" : int(np.ceil(np.asarray(vid_start/tr) + np.asarray(vid_end/tr)) + add_tr),
            "bpress" : df_filt_dic['internal_bpress.rt'][ind]
        }
        # check that movie dimensions are correct before continuing 
        info_checked = check_d(mov_name, info, mov_len_dic)
        run_dic['run-'+str(run)] = info_checked
        run+=1
    

        
## Save ## 
np.save(f'{out_dir}{sub}_behav.npy', sub_dic)
   
    

WARNING, NOT THE SAME MOVIE LENGTH
current cake: -779, TRUE: 99
NOT FIXED SHEEESH
WARNING, NOT THE SAME MOVIE LENGTH
current cake: -879, TRUE: 99
NOT FIXED SHEEESH
WARNING, NOT THE SAME MOVIE LENGTH
current cake: -980, TRUE: 99
NOT FIXED SHEEESH


In [143]:
sub_dic

{'Internal': {'cake': {'run-1': {'start_tr': 11, 'end_tr': 110, 'bpress': []},
   'run-2': {'start_tr': 115, 'end_tr': -664, 'bpress': -1},
   'run-3': {'start_tr': 221, 'end_tr': -658, 'bpress': -1},
   'run-4': {'start_tr': 330, 'end_tr': -650, 'bpress': -1}},
  'brushing': {'run-1': {'start_tr': 7,
    'end_tr': 95,
    'bpress': [58.16177683742717,
     69.29567131632939,
     109.8536771973595,
     119.887692773249]},
   'run-2': {'start_tr': 99,
    'end_tr': 187,
    'bpress': [22.96438198722899,
     56.046382687054574,
     89.40842336323112,
     109.203404228203]},
   'run-3': {'start_tr': 196,
    'end_tr': 284,
    'bpress': [41.452611280139536,
     63.90567263029516,
     97.85164833813906,
     123.43763391533867]},
   'run-4': {'start_tr': 289,
    'end_tr': 377,
    'bpress': [54.40391775779426, 83.24092686595395, 119.14097967278212]}},
  'office': {'run-1': {'start_tr': 7,
    'end_tr': 109,
    'bpress': [52.47381783183664, 97.92982363095507]},
   'run-2': {'start_

In [ ]:
#### sub 015 ####

In [83]:
""" 
df_filt_dic['run_tr_btn.started'] is the time that the fourth TR **started** for each of the 6 runs.
So, I need to add 3 TRs to the initial start time to adjust for each run 
"""
for ind, vid in enumerate(df_filt_dic['run_tr_btn.started']):
    # this is a new video # 
    # set start time for the current run ## 
    ## If new video set the 
    if vid != "-1": 
        start_time = vid
        run_dic = {}
        run = 1
    # Set dictionary key to the current name of the movie after four iterations
    if ((ind + 1) % 4 == 0): 
        cond_dic[label_dic['movies'][ind][:-9]] = run_dic
    
    # set movie name for each iteration
    mov_name = label_dic['movies'][ind][:-9]
    
    # Set dictionary key to internal or external
    ## manuallly hack for sub-#015 **************************************
    if ((ind + 1)== 12 or (ind + 1)== 20): 
        sub_dic[label_dic['condition'][ind]] = cond_dic
        cond_dic = {}
                
    # Check external to see if the current index is null # 
    if df_filt_dic['ex_video_Onset'][ind] != '-1':
        # Start 
        vid_start = df_filt_dic['ex_video_Onset'][ind] - start_time
    
        # End # 
        vid_end = df_filt_dic['fixation_cross.started'][ind] - df_filt_dic['ex_video_Onset'][ind]
        #print(f'{mov_name} starts {vid_start} and \n ends at {vid_end}')
        
        # Dictionary # 
        info = { 
            "start_tr" : int(np.ceil(vid_start/tr) + add_tr),
            "end_tr" : int(np.ceil(np.asarray(vid_start/tr) + np.asarray(vid_end/tr)) + add_tr),
            "bpress" : df_filt_dic['external_bpress.rt'][ind]
        }
        info_checked = check_d(mov_name, info, mov_len_dic)
        run_dic['run-'+str(run)] = info_checked
        run+=1
        
    else:
        #print("INTERNAL")
        # round, add 4 TRs to account for magnet stablization
        vid_start = df_filt_dic['in_video_Onset'][ind] - start_time
        
        # End # 
        vid_end = df_filt_dic['fixation_cross.started'][ind] - df_filt_dic['in_video_Onset'][ind]
        
        # Dictionary # 
        info = { 
            "start_tr" : int(np.ceil(vid_start/tr) + add_tr),
            "end_tr" : int(np.ceil(np.asarray(vid_start/tr) + np.asarray(vid_end/tr)) + add_tr),
            "bpress" : df_filt_dic['internal_bpress.rt'][ind]
        }
        # check that movie dimensions are correct before continuing 
        info_checked = check_d(mov_name, info, mov_len_dic)
        run_dic['run-'+str(run)] = info_checked
        run+=1
    

sub_dic['mov_order'] = mov_prez

## Save ## 
np.save(f'{out_dir}{sub}_behav.npy', sub_dic)
   
    

WARNING, NOT THE SAME MOVIE LENGTH
current office: 101, TRUE: 102
end TR begin 644
end TR END 645
FIXED


In [67]:
sub

'sub-015'

In [81]:
sub_dic['Internal']['office']['run-4']

{'start_tr': 544,
 'end_tr': 646,
 'bpress': [16.385534417815506,
  17.635418847901747,
  19.16852821991779,
  27.998512094840407,
  30.69953189883381,
  37.27359105483629,
  40.54254425386898,
  48.132528573973104,
  51.230516358977184,
  53.65551281091757,
  59.7695485688746,
  65.9475232749246,
  70.56652070884593,
  77.390599564882,
  79.64654505997896,
  89.49155385489576,
  91.78963169781491,
  97.40754399099387,
  103.81452113785781,
  109.40254392102361,
  111.83454532292672,
  118.45555573282763,
  118.88852498400956,
  126.47461053892039,
  132.85956759098917,
  140.27562717790715,
  149.48554315092042]}

In [73]:
sub_dic

{'Internal': {'brushing': {'run-1': {'start_tr': 16,
    'end_tr': 104,
    'bpress': [123.01615735399537]},
   'run-2': {'start_tr': 114, 'end_tr': 202, 'bpress': [87.38443873706274]},
   'run-3': {'start_tr': 209, 'end_tr': 297, 'bpress': [52.36927381600253]},
   'run-4': {'start_tr': 306,
    'end_tr': 394,
    'bpress': [36.139857335947454, 89.79188831395004, 116.94582569296472]}},
  'office': {'run-1': {'start_tr': 7,
    'end_tr': 109,
    'bpress': [98.76934708002955, 143.3823622589698]},
   'run-2': {'start_tr': 116, 'end_tr': 218, 'bpress': -1},
   'run-3': {'start_tr': 224, 'end_tr': 326, 'bpress': [128.5890760840848]},
   'run-4': {'start_tr': 333, 'end_tr': 435, 'bpress': [103.0743074020138]}},
  'cake': {'run-1': {'start_tr': 6,
    'end_tr': 105,
    'bpress': [71.1153376440052]},
   'run-2': {'start_tr': 111,
    'end_tr': 210,
    'bpress': [59.18614917306695, 89.51017270598095, 114.6201694670599]},
   'run-3': {'start_tr': 220,
    'end_tr': 319,
    'bpress': [64.8056

# Actual movie length # 

In [124]:
print('oragami', int(np.ceil(sub_dic['External']['oragami']['run-1']['end_tr'] - 
                  sub_dic['External']['oragami']['run-1']['start_tr'])))

print('shrek', int(np.ceil(sub_dic['External']['shrek']['run-1']['end_tr'] - 
                  sub_dic['External']['shrek']['run-1']['start_tr'])))

print('sherlock', int(np.ceil(sub_dic['External']['sherlock']['run-1']['end_tr'] - 
                  sub_dic['External']['sherlock']['run-1']['start_tr'])))

print('brushing', int(np.ceil(sub_dic['Internal']['brushing']['run-1']['end_tr'] - 
                  sub_dic['Internal']['brushing']['run-1']['start_tr'])))

print('cake', int(np.ceil(sub_dic['Internal']['cake']['run-1']['end_tr'] - 
                  sub_dic['Internal']['cake']['run-1']['start_tr'])))

print('office', int(np.ceil(sub_dic['Internal']['office']['run-1']['end_tr'] - 
                  sub_dic['Internal']['office']['run-1']['start_tr'])))


oragami 82
shrek 90
sherlock 98
brushing 88
cake 99
office 102


In [114]:
int(np.ceil(sub_dic['External']['shrek']['run-2']['end_tr'] - sub_dic['External']['shrek']['run-2']['start_tr']))


90

In [ ]:
int(np.ceil(sub_dic['External']['shrek']['run-2']['end_tr'] - sub_dic['External']['shrek']['run-2']['start_tr']))


In [120]:
sub_dic['Internal']

{'brushing': {'run-1': {'start_tr': 8.0,
   'end_tr': 95.25030759527999,
   'bpress': [22.620965503971092,
    31.368991186027415,
    44.11598844395485,
    71.12596841098275,
    92.64596748293843,
    128.98696864396334]},
  'run-2': {'start_tr': 105.0,
   'end_tr': 192.26043080266874,
   'bpress': [17.019527195952833,
    36.672546693938784,
    66.23754048696719,
    89.41052141203545,
    96.62452668801416,
    107.39751572802197,
    119.0815233780304]},
  'run-3': {'start_tr': 198.0,
   'end_tr': 285.2477625979424,
   'bpress': [18.70805731206201,
    56.77816000301391,
    66.50906278402545,
    88.03503432299476,
    96.44005894404836,
    117.45607598207425,
    122.70611634699162]},
  'run-4': {'start_tr': 293.0,
   'end_tr': 380.24696105734137,
   'bpress': [15.924424874945544,
    28.45444866793696,
    51.28741743799765,
    57.184410081012174,
    64.32543352094945,
    87.84445977292489,
    108.55342765094247,
    123.78744827397168]}},
 'cake': {'run-1': {'start_tr':

In [112]:
sub_dic['External']['shrek']

{'run-1': {'start_tr': 5.0,
  'end_tr': 94.17605273068602,
  'bpress': [46.85239376197569]},
 'run-2': {'start_tr': 100.0,
  'end_tr': 189.17533736598398,
  'bpress': [92.0152772499714]},
 'run-3': {'start_tr': 195.0,
  'end_tr': 284.17362977595377,
  'bpress': [82.80505507998168]},
 'run-4': {'start_tr': 292.0, 'end_tr': 381.16124898730806, 'bpress': -1}}

In [413]:
pd.DataFrame(np.column_stack((tr_vid_start_ind, tr_vid_end_ind, np.round(tr_vid_len))), columns ='')

,0,1,2
0,11.0,116.0,97.0
1,116.0,221.0,97.0
2,218.0,323.0,97.0
3,322.0,427.0,97.0
4,7.0,112.0,82.0
5,93.0,198.0,82.0
6,179.0,284.0,82.0
7,278.0,383.0,82.0
8,5.0,110.0,89.0
9,100.0,205.0,89.0


In [357]:
df_filt

,set1,set2,run_tr_btn.started,fixation_cross.started,ex_video_Onset,external_bpress.started,tr_external.started,in_video_Onset,internal_bpress.started,run_tr_btn.rt,external_bpress.rt,tr_external.rt,internal_bpress.rt,internal_TR.rt
3,movie_stim/brushing_edit.mp4,movie_stim/sherlock_edit.mp4,801.3,963.464,817.773,817.799,817.799,-1,-1,[1.4717830549925566],"[25.759901527082548, 72.17591777001508, 124.31...","[1.471425767056644, 2.971421087044291, 4.47140...",-1,-1
4,movie_stim/brushing_edit.mp4,movie_stim/sherlock_edit.mp4,-1,1120.96,975.266,975.314,975.314,-1,-1,-1,"[27.682404651888646, 90.31649919389747]","[1.4480308439815417, 2.9480271809734404, 4.448...",-1,-1
5,movie_stim/brushing_edit.mp4,movie_stim/sherlock_edit.mp4,-1,1273.96,1128.26,1128.3,1128.3,-1,-1,-1,"[89.64043311704881, 99.03738745406736, 118.120...","[1.4559253229526803, 2.955979263992049, 4.4560...",-1,-1
6,movie_stim/brushing_edit.mp4,movie_stim/sherlock_edit.mp4,-1,1429.94,1284.25,1284.29,1284.29,-1,-1,-1,"[47.35356301290449, 53.99557503696997, 96.0365...","[1.4531753819901496, 2.95315684506204, 4.45317...",-1,-1
7,movie_stim/cake_edit.mp4,movie_stim/oragami_edit.mp4,1567.64,1700.49,1578.12,1578.16,1578.16,-1,-1,[1.4762178799137473],"[69.38565285003278, 75.61968014005106, 97.6567...","[1.4562477499712259, 2.9562702720286325, 4.456...",-1,-1
8,movie_stim/cake_edit.mp4,movie_stim/oragami_edit.mp4,-1,1829.49,1707.11,1707.15,1707.15,-1,-1,-1,"[25.795023905928247, 39.642178528010845, 56.05...","[1.4526776559650898, 2.9526874989969656, 4.452...",-1,-1
9,movie_stim/cake_edit.mp4,movie_stim/oragami_edit.mp4,-1,1958.48,1836.1,1836.15,1836.15,-1,-1,-1,"[30.341299324994907, 31.473302444908768, 46.65...","[1.447837291052565, 2.9478418059879914, 4.4478...",-1,-1
10,movie_stim/cake_edit.mp4,movie_stim/oragami_edit.mp4,-1,2106.96,1984.6,1984.63,1984.63,-1,-1,-1,"[36.14925584604498, 52.16324320703279, 62.3872...","[1.4588406269904226, 2.9588877889327705, 4.458...",-1,-1
11,movie_stim/office_edit.mp4,movie_stim/shrek_edit.mp4,2207.83,2349.09,2215.31,2215.35,2215.35,-1,-1,[1.481007851078175],[46.85239376197569],"[1.4588412198936567, 2.9588246719213203, 4.458...",-1,-1
12,movie_stim/office_edit.mp4,movie_stim/shrek_edit.mp4,-1,2491.59,2357.81,2357.84,2357.84,-1,-1,-1,[92.0152772499714],"[1.4579196309205145, 2.9578250059857965, 4.457...",-1,-1
